# 유흥업소(경기/인천) 데이터 전처리

In [ ]:
import pandas as pd

In [ ]:
alcohol_gg = pd.read_csv('data/유흥업소 관련/유흥주점업소/fulldata_07_23_02_P_유흥주점영업.csv',
                         encoding='cp949')

alcohol_gg.head()

In [ ]:
alcohol_gg.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보x(epsg5174)',
       '좌표정보y(epsg5174)', '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명',
       '급수시설구분명', '총직원수', '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수',
       '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식',
       '홈페이지', 'Unnamed: 47'],
      dtype='object')

In [ ]:
# 필요한 컬럼만 필터링
alcohol_gg01 = alcohol_gg[['상세영업상태명','도로명전체주소','사업장명']]

alcohol_gg01

,상세영업상태명,도로명전체주소,사업장명
0,영업,"광주광역시 광산구 임방울대로801번길 51, 2층 (쌍암동)",놀자룸소주방
1,영업,"광주광역시 광산구 첨단중앙로170번길 36 (쌍암동,(지상 3층))",나비노래홀
2,영업,"광주광역시 광산구 첨단중앙로170번길 36, 2층 (쌍암동)",도드리
3,영업,"광주광역시 광산구 첨단중앙로152번길 81 (쌍암동,(2층))",프러포즈음악홀
4,영업,"광주광역시 광산구 임방울대로825번길 21-11 (쌍암동,(지하1층))",더블유(W)
...,...,...,...
53142,폐업,"경기도 시흥시 중심상가로 332, 2층 일부호 (정왕동)",프라이빗클럽
53143,폐업,NaN,휴가호
53144,폐업,"대구광역시 중구 동성로3길 58-15, 3층,중3층 (공평동)",클럽 박스
53145,폐업,"인천광역시 미추홀구 경원대로 853, 삼미쇼핑 3층 302호 (주안동)",금강산 캬바레


시군구 변환

In [ ]:
alcohol_gg01['구'] = alcohol_gg01['도로명전체주소'].str.extract(r'(\w+구)')

# apply와 lambda를 사용하여 '시도' 컬럼 채우기
def extract_sido(address):
    if pd.isna(address):
        return None
    elif '인천광역시' in address:
        return '인천광역시'
    elif '경기도' in address:
        return '경기도'
    else:
        return None

alcohol_gg01['시도'] = alcohol_gg01['도로명전체주소'].apply(extract_sido)

alcohol_gg01

In [ ]:
# 필요없는 컬럼 삭제
alcohol_icgg = alcohol_gg01.dropna(subset='시도')

In [ ]:
alcohol_icgg

,상세영업상태명,도로명전체주소,사업장명,구,시도
1313,영업,인천광역시 옹진군 백령면 백령로 298-1,장미노래클럽,NaN,인천광역시
1314,영업,인천광역시 옹진군 영흥면 영흥남로 307,빠샤 노래주점,NaN,인천광역시
1315,영업,인천광역시 옹진군 영흥면 영흥북로35번길 21,뉴 차차차 노래클럽,NaN,인천광역시
1316,영업,인천광역시 옹진군 백령면 백령로 282-1,퍼펙트,NaN,인천광역시
1317,영업,"인천광역시 옹진군 백령면 백령로 287, 1층",텔레파시,NaN,인천광역시
...,...,...,...,...,...
53124,폐업,"인천광역시 서구 길주로 79, 지하B01,B301호 (석남동)",쿵텝(KungThep),서구,인천광역시
53135,폐업,"경기도 포천시 소흘읍 솔모루로 55, 5층",7090라이브룸,NaN,경기도
53136,폐업,"경기도 파주시 소리천로 25, 유은타워7차 317호 (야당동)",아리라이브 7080,NaN,경기도
53142,폐업,"경기도 시흥시 중심상가로 332, 2층 일부호 (정왕동)",프라이빗클럽,NaN,경기도


In [ ]:
# 영업중인 데이터만 필터링
open = alcohol_icgg['상세영업상태명'] == '영업'

alcohol_icgg = alcohol_icgg[open]

In [ ]:
si_gun_gu_list = [
    '동구', '남구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군',
    '수원시', '성남시', '의정부시', '안양시', '부천시', '광명시', '평택시', '동두천시', '안산시', '고양시',
    '과천시', '구리시', '남양주시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시', '파주시',
    '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군', '가평군', '양평군', '중구'
]

def extract_si_gun_gu(address):
    if pd.isna(address):  # 주소가 NaN일 경우
        return None
    for si_gun_gu in si_gun_gu_list:
        if si_gun_gu in address:
            return si_gun_gu
    return None  # 시군구 리스트에 해당하는 값이 없는 경우

alcohol_icgg['시군구2'] = alcohol_icgg['도로명전체주소'].apply(extract_si_gun_gu)

alcohol_icgg


In [ ]:
alcohol_icgg['시군구2'].unique()

array(['옹진군', '중구', '미추홀구', '서구', '안산시', '구리시', '성남시', '동두천시', '광주시',
       '가평군', '광명시', '의정부시', '파주시', '양주시', '여주시', '오산시', '이천시', '용인시',
       '평택시', '김포시', '동구', '부평구', '양평군', '남양주시', '계양구', '시흥시', '화성시',
       '수원시', '안성시', '안양시', '강화군', '포천시', '하남시', '의왕시', '연천군', '부천시',
       '연수구', '군포시', '고양시'], dtype=object)

데이터 저장

In [ ]:
alcohol_icgg.to_csv('인천경기_유흥업소.csv', index=False, encoding='utf-8-sig')